In [2]:
import json
from all_def import DomClickApi
import requests
import pandas as pd
from multiprocessing import *
from bs4 import BeautifulSoup
from queue import Queue
import traceback
from threading import Thread
from datetime import datetime
import numpy as np

In [3]:
dca = DomClickApi()

In [4]:
df = pd.read_csv("moscow.csv").drop(labels=["id"], axis=1)
df

,address,price,floor,total_floor,rooms,square,city,remont,balcon,uid,view,year,type_perec,hot_water,fundament,kitchen
0,"Выборгская улица, 8",10680000,1,9,1,35.0,Москва,standard,1,2056866171,garden,NaN,NaN,NaN,NaN,NaN
1,"улица Гурьянова, 9",12990000,3,12,1,40.5,Москва,standard,1,2058752717,garden,NaN,NaN,NaN,NaN,NaN
2,"Ленинградское шоссе, 13к2",10400000,5,5,1,30.0,Москва,standard,1,2056577025,garden,NaN,NaN,NaN,NaN,NaN
3,"Восточная улица, 2к3",12100000,3,8,1,31.1,Москва,standard,1,2059898775,garden,NaN,NaN,NaN,NaN,NaN
4,"Нагорная улица, 21к1",16000000,8,15,1,43.1,Москва,standard,1,2059873457,garden,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11330,"3-й Крутицкий переулок, 11",53000000,8,19,4,116.7,Москва,design,2,2059807526,street,NaN,NaN,NaN,NaN,NaN
11331,"Мичуринский проспект, 11к1",55000000,6,17,4,132.0,Москва,design,2,2058216516,street,NaN,NaN,NaN,NaN,NaN
11332,"улица Дзержинского, 6",18600000,4,16,4,132.0,дачный посёлок Кокошкино,well_done,2,1523532836,street,NaN,NaN,NaN,NaN,NaN
11333,"Профсоюзная улица, 104",46500000,14,17,4,137.2,Москва,well_done,2,2058030172,street,NaN,NaN,NaN,NaN,NaN


In [6]:
def get_extra_data(ui):
    dca = DomClickApi()
    res = dca.get(f"https://domclick.ru/card/sale__flat__{ui}", params={})
    soup = BeautifulSoup(res.content, 'html.parser')
    #material_wall = soup.find('li', {'data-e2e-id': 'Материал стен'}).find('span', {'data-e2e-id': 'Значение'}).text
    p1 = soup.find('li', {'data-e2e-id': 'Год постройки'})
    year = p1.find('span', {'data-e2e-id': 'Значение'}).text if p1 else 0 

    p2 = soup.find('li', {'data-e2e-id': 'Тип фундамента'})
    fundament = p2.find('span', {'data-e2e-id': 'Значение'}).text if p2 else 0

    p3 = soup.find('li', {'data-e2e-id': 'Тип перекрытий'})
    type_perec = p3.find('span', {'data-e2e-id': 'Значение'}).text if p3 else 0

    p4 = soup.find('li', {'data-e2e-id': 'Горячее водоснабжение'})
    hot_water = p4.find('span', {'data-e2e-id': 'Значение'}).text if p4 else 0

    p5 = soup.find('li', {'data-e2e-id': 'Кухня'})
    kitchen = p5.find('span', {'data-e2e-id': 'Значение'}).text if p5 else 0
    return year, type_perec, hot_water, fundament, kitchen


In [7]:
print(get_extra_data(2058968711))

('2022', 'Железобетонный', 'Поквартирный котел', 0, '9,8\xa0м2')


In [120]:
df = np.array_split(df, 17)


c:\Users\danii\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [8]:
from math import ceil

In [9]:
import warnings 
warnings.filterwarnings("ignore")

In [10]:
def extra_data():
        cnt = 0
        for index, row in df.iterrows():
            cnt += 1
            uid = row['uid']
            b, y, s, u, t = get_extra_data(uid)
            df.loc[index, 'year'] = b
            df.loc[index, 'type_perec'] = y
            df.loc[index, 'hot_water'] = s
            df.loc[index, 'fundament'] = u
            df.loc[index, 'kitchen'] = t
            print("Working... ", str(cnt) + '/' + '11335', ' ', ceil((cnt*100) / 11335), "%", end='\r')

In [11]:
extra_data()

In [118]:
df.to_csv("final_data.csv")

OK


In [113]:
result
#result.to_csv("mau.csv")

,address,price,floor,total_floor,rooms,square,city,remont,balcon,uid,view,year,type_perec,hot_water,fundament,kitchen
0,"бульвар Гусева, 57",4150000,7,14,1,38.7,Тверь,standard,1,2058968711,garden,NaN,NaN,NaN,NaN,NaN
1,"улица Благоева, 3к3",3320000,1,5,1,33.8,Тверь,standard,1,2058538048,garden,NaN,NaN,NaN,NaN,NaN
2,"улица Хромова, 14",2850000,3,3,1,31.0,Тверь,standard,1,1795881805,garden,NaN,NaN,NaN,NaN,NaN
3,"улица Коробкова, 17",3200000,2,5,1,34.0,Тверь,standard,1,2059250631,garden,NaN,NaN,NaN,NaN,NaN
4,"улица С.Я. Лемешева, 10",4450000,7,16,1,37.7,Тверь,standard,1,2056021230,garden,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15482,"проспект Непокорённых, 49к2",20500000,7,16,4,135.0,Санкт-Петербург,design,2,1870651187,street,NaN,NaN,NaN,NaN,NaN
15483,"Морской проспект, 33",98900000,2,7,4,241.8,Санкт-Петербург,design,2,1525960658,street,NaN,NaN,NaN,NaN,NaN
15484,"Ушаковская набережная, 3к2",85000000,13,17,4,156.0,Санкт-Петербург,design,2,2059035935,street,NaN,NaN,NaN,NaN,NaN
15485,"Кемская улица, 14",66500000,2,7,4,124.7,Санкт-Петербург,design,2,1633885501,street,NaN,NaN,NaN,NaN,NaN
